In [1]:
import pandas as pd 
import matplotlib as plt
import seaborn as sns
import nltk
from vaderSentiment import vaderSentiment as vader 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm
pd.options.display.max_columns = 999


In [2]:
analyzer = SentimentIntensityAnalyzer()

In [3]:
vad = ['review_vader_'+ str(x+1) for x in range(260)]
lis = ['features'+str(x+1) for x in range(260)]
nums = [x for x in range(1,261)]
rva = ['review_'+ str(x+1) for x in range(260)]

In [4]:
df = pd.read_csv('../../Data/Fullsets/cleanedBronxfull.csv', lineterminator='\n')

In [5]:
df.head()

,Unnamed: 0,city,rating,restaurant_id,restaurant_name,review_count,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,review_10,review_11,review_12,review_13,review_14,review_15,review_16,review_17,review_18,review_19,review_20,review_21,review_22,review_23,review_24,review_25,review_26,review_27,review_28,review_29,review_30,review_31,review_32,review_33,review_34,review_35,review_36,review_37,review_38,review_39,review_40,review_41,review_42,review_43,review_44,review_45,review_46,review_47,review_48,review_49,review_50,review_51,review_52,review_53,review_54,review_55,review_56,review_57,review_58,review_59,review_60,review_61,review_62,review_63,review_64,review_65,review_66,review_67,review_68,review_69,review_70,review_71,review_72,review_73,review_74,review_75,review_76,review_77,review_78,review_79,review_80,review_81,review_82,review_83,review_84,review_85,review_86,review_87,review_88,review_89,review_90,review_91,review_92,review_93,review_94,review_95,review_96,review_97,review_98,review_99,review_100,review_101,review_102,review_103,review_104,review_105,review_106,review_107,review_108,review_109,review_110,review_111,review_112,review_113,review_114,review_115,review_116,review_117,review_118,review_119,review_120,review_121,review_122,review_123,review_124,review_125,review_126,review_127,review_128,review_129,review_130,review_131,review_132,review_133,review_134,review_135,review_136,review_137,review_138,review_139,review_140,review_141,review_142,review_143,review_144,review_145,review_146,review_147,review_148,review_149,review_150,review_151,review_152,review_153,review_154,review_155,review_156,review_157,review_158,review_159,review_160,review_161,review_162,review_163,review_164,review_165,review_166,review_167,review_168,review_169,review_170,review_171,review_172,review_173,review_174,review_175,review_176,review_177,review_178,review_179,review_180,review_181,review_182,review_183,review_184,review_185,review_186,review_187,review_188,review_189,review_190,review_191,review_192,review_193,review_194,review_195,review_196,review_197,review_198,review_199,review_200,review_201,review_202,review_203,review_204,review_205,review_206,review_207,review_208,review_209,review_210,review_211,review_212,review_213,review_214,review_215,review_216,review_217,review_218,review_219,review_220,review_221,review_222,review_223,review_224,review_225,review_226,review_227,review_228,review_229,review_230,review_231,review_232,review_233,review_234,review_235,review_236,review_237,review_238,review_239,review_240,review_241,review_242,review_243,review_244,review_245,review_246,review_247,review_248,review_249,review_250,review_251,review_252,review_253,review_254,review_255,review_256,review_257,review_258,review_259,review_260
0,0,Bronx,4.0,ba-MPbYnM6T30RNgPdUvtQ,Shanghai Red Asian Latin Bistro,54,Ok so imma start with a Thank You to my waitre...,I was really conflicted in writing this review...,"Note: This is a review for delivery, not dinin...","Shanghai's Sunday brunch was alot of fun, but ...",I was in the area waiting for a few friends an...,This place was definitely amazing. Great atmos...,Been here twice now. Cute spot. Servers are ve...,Conveniently located across the street from #6...,Been waiting for this establishment to open an...,Been waiting for this establishment to open an...,"Nice Place, Great Drinks just watch out for th...",Just came here on a Friday at 5 pm and caught ...,Came here without making reservations for my s...,If your like me and you read the reviews to de...,Definitely not impressed with this place. It's...,Stay away. My shrimp tasted of ammonia. Very d...,"As soon as I walked in, I was greeted by the h...","As soon as I walked in, I was greeted by the h...",Came back for brunch on June 2nd. We had a gre...,"I only came for happy hour, but they have good...",Just came from this wonderful place!! With my ...,This was my wife and I first time coming to th.

In [6]:
df.tail(16)

,Unnamed: 0,city,rating,restaurant_id,restaurant_name,review_count,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,review_10,review_11,review_12,review_13,review_14,review_15,review_16,review_17,review_18,review_19,review_20,review_21,review_22,review_23,review_24,review_25,review_26,review_27,review_28,review_29,review_30,review_31,review_32,review_33,review_34,review_35,review_36,review_37,review_38,review_39,review_40,review_41,review_42,review_43,review_44,review_45,review_46,review_47,review_48,review_49,review_50,review_51,review_52,review_53,review_54,review_55,review_56,review_57,review_58,review_59,review_60,review_61,review_62,review_63,review_64,review_65,review_66,review_67,review_68,review_69,review_70,review_71,review_72,review_73,review_74,review_75,review_76,review_77,review_78,review_79,review_80,review_81,review_82,review_83,review_84,review_85,review_86,review_87,review_88,review_89,review_90,review_91,review_92,review_93,review_94,review_95,review_96,review_97,review_98,review_99,review_100,review_101,review_102,review_103,review_104,review_105,review_106,review_107,review_108,review_109,review_110,review_111,review_112,review_113,review_114,review_115,review_116,review_117,review_118,review_119,review_120,review_121,review_122,review_123,review_124,review_125,review_126,review_127,review_128,review_129,review_130,review_131,review_132,review_133,review_134,review_135,review_136,review_137,review_138,review_139,review_140,review_141,review_142,review_143,review_144,review_145,review_146,review_147,review_148,review_149,review_150,review_151,review_152,review_153,review_154,review_155,review_156,review_157,review_158,review_159,review_160,review_161,review_162,review_163,review_164,review_165,review_166,review_167,review_168,review_169,review_170,review_171,review_172,review_173,review_174,review_175,review_176,review_177,review_178,review_179,review_180,review_181,review_182,review_183,review_184,review_185,review_186,review_187,review_188,review_189,review_190,review_191,review_192,review_193,review_194,review_195,review_196,review_197,review_198,review_199,review_200,review_201,review_202,review_203,review_204,review_205,review_206,review_207,review_208,review_209,review_210,review_211,review_212,review_213,review_214,review_215,review_216,review_217,review_218,review_219,review_220,review_221,review_222,review_223,review_224,review_225,review_226,review_227,review_228,review_229,review_230,review_231,review_232,review_233,review_234,review_235,review_236,review_237,review_238,review_239,review_240,review_241,review_242,review_243,review_244,review_245,review_246,review_247,review_248,review_249,review_250,review_251,review_252,review_253,review_254,review_255,review_256,review_257,review_258,review_259,review_260
611,966,New Rochelle,4.0,HwNnlq3kFeDnLrHq0LNU2Q,Juicy Chicken Peruvian Rotisserie Bar & Grill,25,One of the BEST Lomo Saltado dishes I've ever ...,One word : DELICIOSO ! Just walking through t...,From the reviews I was expecting much better. ...,What a great spot. Stopped in for lunch on the...,I come here almost weekly. I've had my fair sh...,"The food is okay to me, my husband loved it, g...",-Cheecha (purple drink) 9/10-Arroz chaufa (veg...,"Have tried this place three times, although we...",They don't honor their grubhub or seamless des...,First off I'm a big Pio Pio fan and have been ...,I usually judge a new/Spanish restaurant by t...,The best rotisserie chicken you will find. Bet...,Great food great service nice atmosphere defin...,Been meaning to go try this place since it ope...,The best Peruvian food in Westchester County.....,After hearing and reading the reviews about th...,"I tried this place twice, first experience was...",If you're into freshly made phenomenal empanad...,Normally I go to another restaurant but I am g...,"Called to place an order , asked for chicken a...",Have been waiting for a place like this! THE. ...,I was really hoping that I w

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Columns: 266 entries, Unnamed: 0 to review_260
dtypes: float64(1), int64(2), object(263)
memory usage: 1.3+ MB


In [8]:
df = df.fillna(value=str('none'))

In [9]:
df.head(15)

,Unnamed: 0,city,rating,restaurant_id,restaurant_name,review_count,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,review_10,review_11,review_12,review_13,review_14,review_15,review_16,review_17,review_18,review_19,review_20,review_21,review_22,review_23,review_24,review_25,review_26,review_27,review_28,review_29,review_30,review_31,review_32,review_33,review_34,review_35,review_36,review_37,review_38,review_39,review_40,review_41,review_42,review_43,review_44,review_45,review_46,review_47,review_48,review_49,review_50,review_51,review_52,review_53,review_54,review_55,review_56,review_57,review_58,review_59,review_60,review_61,review_62,review_63,review_64,review_65,review_66,review_67,review_68,review_69,review_70,review_71,review_72,review_73,review_74,review_75,review_76,review_77,review_78,review_79,review_80,review_81,review_82,review_83,review_84,review_85,review_86,review_87,review_88,review_89,review_90,review_91,review_92,review_93,review_94,review_95,review_96,review_97,review_98,review_99,review_100,review_101,review_102,review_103,review_104,review_105,review_106,review_107,review_108,review_109,review_110,review_111,review_112,review_113,review_114,review_115,review_116,review_117,review_118,review_119,review_120,review_121,review_122,review_123,review_124,review_125,review_126,review_127,review_128,review_129,review_130,review_131,review_132,review_133,review_134,review_135,review_136,review_137,review_138,review_139,review_140,review_141,review_142,review_143,review_144,review_145,review_146,review_147,review_148,review_149,review_150,review_151,review_152,review_153,review_154,review_155,review_156,review_157,review_158,review_159,review_160,review_161,review_162,review_163,review_164,review_165,review_166,review_167,review_168,review_169,review_170,review_171,review_172,review_173,review_174,review_175,review_176,review_177,review_178,review_179,review_180,review_181,review_182,review_183,review_184,review_185,review_186,review_187,review_188,review_189,review_190,review_191,review_192,review_193,review_194,review_195,review_196,review_197,review_198,review_199,review_200,review_201,review_202,review_203,review_204,review_205,review_206,review_207,review_208,review_209,review_210,review_211,review_212,review_213,review_214,review_215,review_216,review_217,review_218,review_219,review_220,review_221,review_222,review_223,review_224,review_225,review_226,review_227,review_228,review_229,review_230,review_231,review_232,review_233,review_234,review_235,review_236,review_237,review_238,review_239,review_240,review_241,review_242,review_243,review_244,review_245,review_246,review_247,review_248,review_249,review_250,review_251,review_252,review_253,review_254,review_255,review_256,review_257,review_258,review_259,review_260
0,0,Bronx,4.0,ba-MPbYnM6T30RNgPdUvtQ,Shanghai Red Asian Latin Bistro,54,Ok so imma start with a Thank You to my waitre...,I was really conflicted in writing this review...,"Note: This is a review for delivery, not dinin...","Shanghai's Sunday brunch was alot of fun, but ...",I was in the area waiting for a few friends an...,This place was definitely amazing. Great atmos...,Been here twice now. Cute spot. Servers are ve...,Conveniently located across the street from #6...,Been waiting for this establishment to open an...,Been waiting for this establishment to open an...,"Nice Place, Great Drinks just watch out for th...",Just came here on a Friday at 5 pm and caught ...,Came here without making reservations for my s...,If your like me and you read the reviews to de...,Definitely not impressed with this place. It's...,Stay away. My shrimp tasted of ammonia. Very d...,"As soon as I walked in, I was greeted by the h...","As soon as I walked in, I was greeted by the h...",Came back for brunch on June 2nd. We had a gre...,"I only came for happy hour, but they have good...",Just came from this wonderful place!! With my ...,This was my wife and I first time coming to th.

In [10]:
for j in tqdm(nums):
    lis[j-1] = df[rva[j-1]].apply(lambda x: analyzer.polarity_scores(x)).apply(pd.Series)
    df = pd.concat([df,lis[j-1].apply(pd.Series)],1)
    df.rename(columns={'neg': 'neg'+str(j), 'neu': 'neu'+str(j), 'pos': 'pos'+str(j), 'compound': 'compound'+str(j)}, inplace=True)
    

100%|██████████| 260/260 [03:09<00:00,  3.03it/s]


In [17]:
df.to_csv('vaderBronx.csv')

In [18]:
import _pickle as pickle

In [19]:
filename = 'vaderBronx.p'
with open(filename, 'wb') as filehandler:
    pickle.dump(df, filehandler)

In [15]:
df.drop(columns='Unnamed: 0', inplace=True)

In [16]:
df.head()

,city,rating,restaurant_id,restaurant_name,review_count,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,review_10,review_11,review_12,review_13,review_14,review_15,review_16,review_17,review_18,review_19,review_20,review_21,review_22,review_23,review_24,review_25,review_26,review_27,review_28,review_29,review_30,review_31,review_32,review_33,review_34,review_35,review_36,review_37,review_38,review_39,review_40,review_41,review_42,review_43,review_44,review_45,review_46,review_47,review_48,review_49,review_50,review_51,review_52,review_53,review_54,review_55,review_56,review_57,review_58,review_59,review_60,review_61,review_62,review_63,review_64,review_65,review_66,review_67,review_68,review_69,review_70,review_71,review_72,review_73,review_74,review_75,review_76,review_77,review_78,review_79,review_80,review_81,review_82,review_83,review_84,review_85,review_86,review_87,review_88,review_89,review_90,review_91,review_92,review_93,review_94,review_95,review_96,review_97,review_98,review_99,review_100,review_101,review_102,review_103,review_104,review_105,review_106,review_107,review_108,review_109,review_110,review_111,review_112,review_113,review_114,review_115,review_116,review_117,review_118,review_119,review_120,review_121,review_122,review_123,review_124,review_125,review_126,review_127,review_128,review_129,review_130,review_131,review_132,review_133,review_134,review_135,review_136,review_137,review_138,review_139,review_140,review_141,review_142,review_143,review_144,review_145,review_146,review_147,review_148,review_149,review_150,review_151,review_152,review_153,review_154,review_155,review_156,review_157,review_158,review_159,review_160,review_161,review_162,review_163,review_164,review_165,review_166,review_167,review_168,review_169,review_170,review_171,review_172,review_173,review_174,review_175,review_176,review_177,review_178,review_179,review_180,review_181,review_182,review_183,review_184,review_185,review_186,review_187,review_188,review_189,review_190,review_191,review_192,review_193,review_194,review_195,review_196,review_197,review_198,review_199,review_200,review_201,review_202,review_203,review_204,review_205,review_206,review_207,review_208,review_209,review_210,review_211,review_212,review_213,review_214,review_215,review_216,review_217,review_218,review_219,review_220,review_221,review_222,review_223,review_224,review_225,review_226,review_227,review_228,review_229,review_230,review_231,review_232,review_233,review_234,review_235,review_236,review_237,review_238,review_239,review_240,review_241,review_242,review_243,review_244,review_245,review_246,review_247,review_248,review_249,review_250,review_251,review_252,review_253,review_254,review_255,review_256,review_257,review_258,review_259,review_260,neg1,neu1,pos1,compound1,neg2,neu2,pos2,compound2,neg3,neu3,pos3,compound3,neg4,neu4,pos4,compound4,neg5,neu5,pos5,compound5,neg6,neu6,pos6,compound6,neg7,neu7,pos7,compound7,neg8,neu8,pos8,compound8,neg9,neu9,pos9,compound9,neg10,neu10,pos10,compound10,neg11,neu11,pos11,compound11,neg12,neu12,pos12,compound12,neg13,neu13,pos13,compound13,neg14,neu14,pos14,compound14,neg15,neu15,pos15,compound15,neg16,neu16,pos16,compound16,neg17,neu17,pos17,compound17,neg18,neu18,pos18,compound18,neg19,neu19,pos19,compound19,neg20,neu20,pos20,compound20,neg21,neu21,pos21,compound21,neg22,neu22,pos22,compound22,neg23,neu23,pos23,compound23,neg24,neu24,pos24,compound24,neg25,neu25,pos25,compound25,neg26,neu26,pos26,compound26,neg27,neu27,pos27,compound27,neg28,neu28,pos28,compound28,neg29,neu29,pos29,compound29,neg30,neu30,pos30,compound30,neg31,neu31,pos31,compound31,neg32,neu32,pos32,compound32,neg33,neu33,pos33,compound33,neg34,neu34,pos34,compound34,neg35,neu35,pos35,compound35,neg36,neu36,pos36,compound36,neg37,neu37,pos37,compound37,neg38,neu38,pos38,compound38,neg39,neu39,pos39,compound39,neg40,neu40,pos40,compound40,neg41,neu41,pos41,compound41,neg42,neu42,pos42,compound42,neg43,neu4

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Columns: 1306 entries, Unnamed: 0 to compound260
dtypes: float64(1041), int64(2), object(263)
memory usage: 6.2+ MB


In [13]:
df.tail(16)

Unnamed: 0          city  rating           restaurant_id  \
611         966  New Rochelle     4.0  HwNnlq3kFeDnLrHq0LNU2Q   
612         971      New York     3.5  5qwk0BNZnm6jmq-M0kP3_Q   
613         972  Mount Vernon     4.0  ZhEMIxeTh-FmaqJG51H3JQ   
614         974  New Rochelle     4.0  _WcjDeOZlFmx_2Quh49j2g   
615         975         Bronx     2.5  hmQrfK0Aem0BWhXeZoKuKA   
616         976         Bronx     4.0  zXELxQsoffCoBiIZamoBsg   
617         977         Bronx     3.5  c2_vGr_fqC3MAskfqto7dw   
618         978      New York     4.5  lIE4KHySIenwPbta1JuM8Q   
619         979      New York     4.5  menL_0IVFYfpuhhgvKuCsQ   
620         980      New York     4.0  pFBNTJpy1eWmfu6aQ8e1YA   
621         982         Bronx     2.5  anPARbCY84k_5AmxVaTAJw   
622         985      New York     3.5  WvJzhvru8oenl6rwVIl0TQ   
623         987      New York     4.0  aYS8PnNfRHeJI-vIR6L9vA   
624         993         Bronx     3.0  OZannIZV3kue8SJK7Dc9qQ   
625         997      New York     4.5  SPVgM1vFJYeExIfZ-NmNoQ   
626         999         Bronx     4.0  pbRgb9o3hegZxWeu6wAtHw   

                                   restaurant_name  review_count  \
611  Juicy Chicken Peruvian Rotisserie Bar & Grill            25   
612                                   El Floridita           106   
613                         Gina Marie's Trattoria            23   
614                           Tzatziki Greek Grill           167   
615                                 Parilla Latina            29   
616                                      Gyro King            41   
617                      Brisas Express Restaurant            10   
618                                  Mumbai Masala           179   
619                            Antojitos Mexicanos            15   
620                                    Cuchifritos           162   
621                             Dancing Crane Cafe            10   
622                         Pat'e Palo Bar & Grill            85   
623                                   Make My Fish            33   
624                             Paddy's On the Bay            47   
625                                       Franco's            37   
626                                      El Porton            24   

                                              review_1  \
611  One of the BEST Lomo Saltado dishes I've ever ...   
612  This place has great Cuban food.. i always ord...   
613  What an amazing dinner! The portions were so g...   
614  I have passed this place for the few years the...   
615  it was lunchtime and the tables available were...   
616  Gyro with everything please! I don't frequent ...   
617  Moved to the neighborhood in September and I a...   
618  Just had lunch here with family and thoroughly...   
619  Best Mexican food in the heights. Hands down. ...   
620  Dear Jesus. I haven't had Cuchifritos this goo...   
621  They make their own pizza dough and sauce on s...   
622  I came to Pat'r Palo for my birthday last year...   
623  The fish dinner with two sides of collard gree...   
624  I've come here a handful of times, and each ti...   
625  Their smoothies are great!!! So refreshing and...   
626  Now that my husband and I are somewhat regular...   

                                              review_2  \
611  One word : DELICIOSO !  Just walking through t...   
612  Great spot for late night food and drink and e...   
613  Amazing new Italian restaurant in Westchester!...   
614  Delicious greek food in huge proportions. High...   
615  Just ordered off Uber eats and the food was so...   
616  Gyro King has been a solid place for the hand ...   
617  The restaurant is pretty clean. The waiter was...   
618  Dinner with a side of sexual harassment. #time...   
619  Fresh, fast, and authentic is how i like my ta...   
620  I was walking in the area with my friend, and ...   
621  You know what you're in for when you find your...   
622  I was here specifically on June 17th, had not ...   
6